<a href="https://colab.research.google.com/github/samarthya04/AI_Lab/blob/main/lab8_2205498.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment 8**: Navigation with Multiple Goals

Objective: Solve a problem where multiple goals exist using search algorithms.

Problem Statement: A robot in a grid needs to collect items (goals) before reachinganexit. Each goal has a different priority or cost.

Tasks:
- Use BFS/DFS for simpler scenarios (unweighted goals).
- Implement A* or Uniform Cost Search for weighted scenarios.
- Analyze the trade-offs between path length and goal priority.